# December 15, 2021

https://adventofcode.com/2021/day/15

In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict, deque
from queue import PriorityQueue

In [2]:
def format_data( data_str ):
    return [ [int(x) for x in line] for line in data_str.split("\n") ]

In [3]:
with open("../data/2021/15.txt", "r") as f:
    data_str = f.read()
data_mat = format_data( data_str )

In [4]:
test_str = '''1163751742
1381373672
2136511328
3694931569
7463417111
1319128137
1359912421
3125421639
1293138521
2311944581'''

test_mat = format_data( test_str )
test_mat

[[1, 1, 6, 3, 7, 5, 1, 7, 4, 2],
 [1, 3, 8, 1, 3, 7, 3, 6, 7, 2],
 [2, 1, 3, 6, 5, 1, 1, 3, 2, 8],
 [3, 6, 9, 4, 9, 3, 1, 5, 6, 9],
 [7, 4, 6, 3, 4, 1, 7, 1, 1, 1],
 [1, 3, 1, 9, 1, 2, 8, 1, 3, 7],
 [1, 3, 5, 9, 9, 1, 2, 4, 2, 1],
 [3, 1, 2, 5, 4, 2, 1, 6, 3, 9],
 [1, 2, 9, 3, 1, 3, 8, 5, 2, 1],
 [2, 3, 1, 1, 9, 4, 4, 5, 8, 1]]

# Part 1

In [5]:
class RiskMap:
    def __init__(self, mat):
        self.map = mat
        self.nx = len(mat[0])
        self.ny = len(mat)

    def neighbors(self, pt):
        return ( [ Point( (pt[0]+d[0], pt[1]+d[1]), self ) for d in [(-1,0), (1,0), (0,1), (0,-1)]
                    if 0 <= pt[0]+d[0] < self.nx and 0 <= pt[1]+d[1] < self.ny] )
        
    def risk(self, pt):
        return self.map[pt[1]][pt[0]]
    
    def estimate(self, pt):
        return abs(self.ny-1 - pt[1]) + abs(self.nx-1 - pt[0])
    
class Point:
    # tuple plus risk and estimated distance to end
    def __init__(self, loc, risk_map):
        self.loc = loc
        self.risk = risk_map.risk( loc )
        self.estimate = risk_map.estimate( loc )

    def __getitem__(self, idx):
        return self.loc[idx]
    def __str__(self):
        return str(self.loc[0]) + ":" + str(self.loc[1])
    def __repr__(self):
        return str(self)
    def __eq__(self, other):
        return self.loc == other.loc
    
class Path:
    # path is list of Points
    def __init__(self, path, risk, estimate=0):
        self.path = path
        self.risk = risk
        self.estimate = estimate
        self.priority = risk + estimate
    
    def end(self):
        return self.path[-1]
    
    def __str__(self):
        return f"""{self.priority} = {self.risk} + {self.estimate}? [{", ".join([str(x) for x in self.path])}]"""
    def __repr__(self):
        return str(self)
    def __lt__(self, other):
        return self.priority < other.priority
    def __add__(self, point):
        return Path( self.path + [point], self.risk + point.risk, point.estimate )

In [6]:
def best_path( risk_map, start=(0,0) ):
    start = Point(start, risk_map)
    goal = Point( (risk_map.nx - 1, risk_map.ny - 1), risk_map )

    # dict of reached locations with value == path risk to that point.
    reached = { start.loc : 0 }

    frontier = PriorityQueue()
    frontier.put( Path([start], 0, start.estimate) )

    while not frontier.empty():
        cur = frontier.get()

        # check all the neighboring points
        neighbors = risk_map.neighbors( cur.end() )
        for nn in neighbors:

            if nn == goal:
                # we did it!
                return cur + nn
            
            nn_path_risk = cur.risk + nn.risk

            if nn.loc in reached.keys():
                # We've been here before. Only explore neighbors
                # if the current path beats the previous path
                if nn_path_risk >= reached[nn.loc]:
                    # old path is better. skip the rest of the loop
                    continue

            reached[nn.loc] = nn_path_risk
            nn_path = cur + nn
            frontier.put( nn_path )

    # ruh-roh, we didn't make it
    return None

In [7]:
test = RiskMap(test_mat)
bp = best_path(test)
bp

40 = 40 + 0? [0:0, 0:1, 0:2, 1:2, 2:2, 3:2, 4:2, 5:2, 6:2, 6:3, 7:3, 7:4, 7:5, 8:5, 8:6, 8:7, 8:8, 9:8, 9:9]

In [8]:
data = RiskMap(data_mat)
bp = best_path(data)
bp

366 = 366 + 0? [0:0, 0:1, 0:2, 1:2, 2:2, 2:3, 2:4, 2:5, 2:6, 3:6, 3:7, 4:7, 5:7, 5:8, 5:9, 6:9, 6:10, 7:10, 8:10, 9:10, 9:11, 10:11, 11:11, 11:12, 12:12, 13:12, 14:12, 14:13, 14:14, 14:15, 14:16, 15:16, 16:16, 17:16, 18:16, 18:17, 19:17, 19:18, 19:19, 19:20, 19:21, 19:22, 19:23, 19:24, 20:24, 21:24, 21:25, 21:26, 22:26, 23:26, 24:26, 25:26, 26:26, 27:26, 28:26, 28:27, 28:28, 29:28, 29:29, 29:30, 29:31, 29:32, 30:32, 31:32, 32:32, 32:33, 32:34, 32:35, 32:36, 33:36, 34:36, 34:37, 35:37, 36:37, 37:37, 38:37, 39:37, 39:38, 40:38, 41:38, 41:39, 42:39, 42:40, 42:41, 42:42, 43:42, 44:42, 44:43, 44:44, 45:44, 46:44, 46:45, 46:46, 47:46, 47:47, 47:48, 48:48, 48:49, 48:50, 48:51, 49:51, 50:51, 50:52, 51:52, 52:52, 53:52, 53:53, 54:53, 54:54, 54:55, 54:56, 55:56, 55:57, 55:58, 55:59, 56:59, 57:59, 58:59, 58:60, 58:61, 58:62, 58:63, 58:64, 59:64, 60:64, 61:64, 62:64, 62:65, 62:66, 62:67, 63:67, 64:67, 65:67, 66:67, 66:68, 66:69, 66:70, 66:71, 66:72, 66:73, 66:74, 66:75, 66:76, 66:77, 66:78, 66:79,

# Part 2

expanding map!

In [9]:
class RiskMapExpanded:
    def __init__(self, mat, rep):
        self.map = mat
        self.tile_nx = len(mat[0])
        self.tile_ny = len(mat)
        self.nx = self.tile_nx * rep
        self.ny = self.tile_ny * rep

    def neighbors(self, pt):
        return ( [ Point( (pt[0]+d[0], pt[1]+d[1]), self ) for d in [(-1,0), (1,0), (0,1), (0,-1)]
                if 0 <= pt[0]+d[0] < self.nx and 0 <= pt[1]+d[1] < self.ny ] )
    
    def risk(self, pt):
        x, y = pt[0], pt[1]
        tilex = int(x / self.tile_nx)
        tiley = int(y / self.tile_ny)

        posx = x - tilex * self.tile_nx
        posy = y - tiley * self.tile_ny

        # each tile over or down increases risk by 1
        risk = self.map[posy][posx] + tilex + tiley

        # except that it wraps around from 10 ---> 1
        return ((risk-1) % 9) + 1
    
    def estimate(self, pt):
        return abs(self.ny-1 - pt[1]) + abs(self.nx-1 - pt[0])

In [10]:
test = RiskMapExpanded(test_mat, rep=5)
bp = best_path(test)
bp

315 = 315 + 0? [0:0, 0:1, 0:2, 0:3, 0:4, 0:5, 0:6, 0:7, 0:8, 0:9, 0:10, 0:11, 0:12, 1:12, 2:12, 2:13, 3:13, 3:14, 3:15, 3:16, 4:16, 5:16, 6:16, 7:16, 8:16, 9:16, 9:17, 9:18, 10:18, 11:18, 12:18, 12:19, 13:19, 14:19, 14:20, 14:21, 15:21, 15:22, 16:22, 16:23, 16:24, 16:25, 17:25, 18:25, 19:25, 19:26, 19:27, 19:28, 20:28, 21:28, 22:28, 22:29, 23:29, 24:29, 24:30, 25:30, 26:30, 27:30, 27:31, 27:32, 27:33, 28:33, 29:33, 29:34, 30:34, 31:34, 32:34, 32:35, 32:36, 33:36, 33:37, 34:37, 34:38, 34:39, 35:39, 36:39, 37:39, 37:40, 37:41, 37:42, 37:43, 38:43, 39:43, 40:43, 41:43, 41:44, 41:45, 41:46, 42:46, 42:47, 43:47, 44:47, 45:47, 45:48, 45:49, 46:49, 47:49, 48:49, 49:49]

In [11]:
data = RiskMapExpanded(data_mat, rep=5)
bp = best_path(data)
print(bp.risk)

2829
